# Pairs Trading

Pairs trading is a form of mean-reversion quantitative strategy. In pairs trading, there are two different different equities that are traded and have a very high correlation value associated with each other. The assumption here is that these two equities have a stationary value where they will converge to whenever they begin to diverge. When these equities begin converging to this stationary value, trades are executed related to a short or long position.

Importing packages and retrieving 1 year of stock data using `yfinance`.

In [5]:
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import yfinance as yf

def get_data(tickers):
    data = pd.DataFrame()
    names = list()

    for tick in tickers:
        data = pd.concat([data, pd.DataFrame(yf.download(tick, start=datetime(2022, 10, 27), end=datetime(2023, 10, 27)).iloc[:,4])], axis = 1)
        names.append(tick)

    data.columns = names
    return data

tickers = ["AAPL", "GOOG", "AMD", "GME", "SPY", "NFLX","AMZN", "BA", "WMT","CMG","GS","XOM","NKE","INTC", "META","BRK-B", "MSFT"] 
stock_data = get_data(tickers)

stock_data.tail()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,AAPL,GOOG,AMD,GME,SPY,NFLX,AMZN,BA,WMT,CMG,GS,XOM,NKE,INTC,META,BRK-B,MSFT
Date,,,,,,,,,,,,,,,,,
2023-10-20,171.999863,136.405014,101.809998,13.44,416.845093,400.959991,125.169998,180.039993,52.215137,36.625000,292.006958,107.317192,101.113297,34.345562,308.017029,335.859985,324.236298
2023-10-23,172.119247,137.562164,100.010002,13.44,416.122620,406.839996,126.559998,181.029999,52.955143,36.393200,291.199249,105.742409,101.251167,33.293171,313.366058,336.839996,326.866516
2023-10-24,172.557007,139.776733,101.669998,14.04,419.259949,413.730011,128.559998,182.360001,53.691868,37.101799,291.160309,104.718315,103.585236,34.021000,311.909027,338.630005,328.067505
2023-10-25,170.228912,126.359680,96.059998,13.71,413.242615,411.250000,121.389999,177.729996,53.530708,36.837399,288.805176,104.911552,101.970100,32.289951,298.915741,336.899994,338.131958
2023-10-26,166.040359,123.137604,93.669998,13.50,408.294220,403.540009,119.570000,179.089996,53.205105,36.129398,289.028992,103.955078,98.503479,31.985046,287.758667,336.160004,325.447205
